In [ ]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import os
from geopy.geocoders import Nominatim

#bruker et bibliotek for å finne koordinater basert på input, for å slippe at brukeren må skrive inn koordinater
geolocator = Nominatim(user_agent="miljøanalyse_abel")
location = geolocator.geocode(input("location: "))
print(location.address)
print((location.latitude, location.longitude))

# Tidsperiode og API-parametere
starttid = "20050101"  # YYYYMMDD
sluttid = "20060101"  # YYYYMMDD
type = "daily"
lon_min, lon_max = location.longitude-1.01, location.longitude+1.01
lat_min, lat_max = location.latitude-1.01, location.latitude+1.01

# CSV-fil for samlet data
csv_fil = "lokasjonsdata.csv"

# Sjekk om byen allerede finnes i CSV-filen
if os.path.exists(csv_fil):
    df_tidligere = pd.read_csv(csv_fil, index_col=0, sep="\t", parse_dates=True)
    if location_name in df_tidligere.columns:
        exit(f"Data for {location_name} finnes allerede i filen. Ingen nye data er lagt til.")

# API-forespørsel
url = f"https://power.larc.nasa.gov/api/temporal/{type}/regional?parameters=ALLSKY_SFC_SW_DWN&community=SB&longitude-min={lon_min}&longitude-max={lon_max}&latitude-min={lat_min}&latitude-max={lat_max}&start={starttid}&end={sluttid}&format=JSON"

response = requests.get(url)
if response.status_code == 422:
    exit("Ugyldig forespørsel.")
elif response.status_code == 429:
    exit("For mange forespørsler. Prøv igjen senere.")
elif response.status_code != 200:
    exit(f"Feil ved forespørsel: {response.status_code}")

# Hente ut data fra responsen
data = response.json()["features"][0]["properties"]["parameter"]["ALLSKY_SFC_SW_DWN"]

# Konverter data til en DataFrame
unique_location_name = location_name  # Bruk kun bynavnet som kolonnenavn
df = pd.DataFrame(list(data.values()), index=pd.to_datetime(list(data.keys())), columns=[unique_location_name])

# Lagre eller oppdatere CSV-fil
if os.path.exists(csv_fil):
    # Kombinerer eksisterende data med ny data
    df_sammensatt = pd.concat([df_tidligere, df], axis=1)
else:
    # Ingen tidligere data, bruk kun ny data
    df_sammensatt = df

# Lagre oppdatert DataFrame til CSV
df_sammensatt.to_csv(csv_fil, sep="\t", encoding="utf-8")
print(f"Data lagret i {csv_fil}")

# Plot sammenligning av data
df_sammensatt.columns = [col.split("_")[0] for col in df_sammensatt.columns]  # Fjern tidsperioden fra kolonnenavn
df_sammensatt.plot(figsize=(10, 5))
plt.xlabel("Dag")
plt.ylabel("Solinnstråling (W/m^2)")
plt.title("Sammenligning av solinnstråling over tid")
plt.legend(df_sammensatt.columns)
plt.show()

Lokasjon ikke funnet. Prøv igjen.


KeyboardInterrupt: Interrupted by user

In [19]:
import numpy as np

def average(data):
    return np.mean(data)

def std_dev(data):
    return np.std(data)

def median(data):
    return np.median(data)

print(f"Gjennomsnitt: {average(df[0]):.3f}")
print(f"Standardavvik: {std_dev(df[0]):.3f}")
print(f"Median: {median(df[0]):.3f}")

Gjennomsnitt: 111.454
Standardavvik: 95.107
Median: 89.745
